### Importing all the Libraries

In [ ]:
# !pip install geotext

In [1]:
pysparkzip = "/opt/dse/resources/spark/python/lib/pyspark.zip"
py4jzip = "/opt/dse/resources/spark/python/lib/py4j-0.10.4-src.zip"

In [2]:
# Needed to be able to find pyspark libaries
import sys
sys.path.append(pysparkzip)
sys.path.append(py4jzip)

In [233]:
import pandas
import cassandra
import pyspark
import tweepy
import re
import os
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pattern.en import sentiment, positive
from geotext import GeoText
import unicodedata

### Importing Twitter API keys and tokens

In [4]:
consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']
access_token = os.environ['ACCESS_TOKEN']
access_token_secret = os.environ['ACCESS_TOKEN_SECRET']


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Function to format Spark DataFrames

In [5]:
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

### Creating Tables, Pulling Tweets, and Loading Tables

#### Connect to Cluster

In [6]:
from cassandra.cluster import Cluster

cluster = Cluster(['dse'])
session = cluster.connect()

#### Keyspace 

In [7]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sentimentanalytics
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Setting keyspace 

In [8]:
session.set_keyspace('sentimentanalytics')

In [9]:
tweetTitle = "covid19"

In [10]:
positiveNegative = ["positive", "negative"] 

In [100]:
query = "DROP TABLE sentiment_%s_%s" % (tweetTitle, positiveNegative[0])
session.execute(query)
query = "DROP TABLE sentiment_%s_%s" % (tweetTitle, positiveNegative[1])
session.execute(query)



for emotion in positiveNegative:
    query = "CREATE TABLE IF NOT EXISTS sentiment_%s_%s (twitterid bigint, tweet text, city text, country text,\
    user text, createdat text, hashtagkeys text, source text, retweets int, userstatuscount int, PRIMARY KEY \
    ((twitterid, tweet, city, country, user, hashtagkeys, source, retweets, userstatuscount), createdat)) with \
    clustering order by (createdat desc)" % (tweetTitle, emotion)
    print query
    session.execute(query)


CREATE TABLE IF NOT EXISTS sentiment_covid19_positive (twitterid bigint, tweet text, city text, country text,    user text, createdat text, hashtagkeys text, source text, retweets int, userstatuscount int, PRIMARY KEY     ((twitterid, tweet, city, country, user, hashtagkeys, source, retweets, userstatuscount), createdat)) with     clustering order by (createdat desc)
CREATE TABLE IF NOT EXISTS sentiment_covid19_negative (twitterid bigint, tweet text, city text, country text,    user text, createdat text, hashtagkeys text, source text, retweets int, userstatuscount int, PRIMARY KEY     ((twitterid, tweet, city, country, user, hashtagkeys, source, retweets, userstatuscount), createdat)) with     clustering order by (createdat desc)


In [101]:
# for emotion in positiveNegative:
# #     query = "CREATE TABLE IF NOT EXISTS covid_sentiments1_%s_%s (twitterid bigint, tweet text, place text, user text,\
# #              createdat timestamp, hashtagkeys text, source text, retweets text, userstatuscount text,userscreenname \
# #              text, PRIMARY KEY (twitterid))" % (tweetTitle, emotion)
#     query = "CREATE TABLE IF NOT EXISTS sentimentofCountry_%s_%s (twitterid bigint, country text, tweet text, city text, \
#     user text, createdat text, hashtagkeys text, source text, retweets int, userstatuscount int, PRIMARY KEY \
#     ((country), createdat)) with \
#     clustering order by (createdat desc)" % (tweetTitle, emotion)
#     print query
#     session.execute(query)

In [102]:
'''Function to CleanUp Each Tweet before if is inserted into Cassandra.
    Removing: 
        * emojis 
        * flags 
        * special characters 
        * URL's 
        * RT (for Retweet)
'''

def cleanUpTweet(tweet):
    if tweet == None:
        return 'unknown'
    
    emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"
    u"(\ud83c[\udf00-\uffff])|"  
    u"(\ud83d[\u0000-\uddff])|" 
    u"(\ud83d[\ude80-\udeff])|"  
    u"(\ud83c[\udde0-\uddff])" 
    "+", flags=re.UNICODE)

#     removeSpecial = re.compile ('[\n|#|@|!|.|?|,|\"]')
    removeSpecial = re.compile ('[\n|@|!|.|?|,|\"]')
    removeHttp = re.compile("http\S+ | https\S+")
    removeRetweet = re.compile("RT")
    
    noemoji = emoji_pattern.sub(r'', tweet)
    nospecial = removeSpecial.sub(r'', noemoji)
    nohttp = removeHttp.sub(r'', nospecial)
    noretweet = removeRetweet.sub(r'', nohttp)
    
    cleanTweet=noretweet
#     cleanTweet = ''.join([x for x in cleanTweet if x.isalnum()])
#     cleanTweet = re.sub(r"[^a-zA-Z0-9:.,@#&]+", ' ', cleanTweet)
    cleanTweet = unicodedata.normalize('NFKD', cleanTweet).encode('ascii', 'ignore')
    return cleanTweet


def cleanUpPlace(tweet):
    if tweet == None:
        return 'unknown'
    tweet = tweet.title()
    places = GeoText(tweet)
    cities = list(places.cities)
    countries = list(places.countries)
    city, country = 'unknown', 'unknown'
    if len(cities) > 0:
        city = str(cities[0])
    if len(countries) > 0:
        country = str(countries[0])
    return (city, country)

def cleanHashtags(hashtag_dict):
    hashtags = []
    for h_dict in hashtag_dict:
        for h in h_dict.values():
            if type(h) == unicode:
                decoded = unicodedata.normalize('NFKD', h).encode('ascii', 'ignore')
                hashtags.append(decoded)
    
    if len(hashtags) == 0:
        hashtags = ['unknown']
    
    hashtags = ','.join(hashtags)
    return hashtags

In [103]:
# pulling data from twitter and inserting into cassandra
titles = ['Covid19']#, 'Covid-19', 'Coronavirus', 'StayHomeStaySafe', 'StayHome', 'LockdownNow']
pos_tags = [' recovery', ' vaccinated']#, ' relax', ' strong', ' cheers', ' blessed', ' care', 
                                       #' QuarantineandChill', ' SocialDistancing']
neg_tags = [' deaths', ' frustrated']#, ' terrible', ' unfocussed', ' fear', ' no vaccine', ' stress', 
                                     #' anxiety', ' worry', ' hardship', ' Covidiots']

pos_terms = [x+y for x in titles for y in pos_tags]
neg_terms = [x+y for x in titles for y in neg_tags]

searchTerms = [pos_terms, neg_terms]


for emotion in positiveNegative:
    print emotion
    public_tweets = 0
    query = "INSERT INTO sentiment_%s_%s (twitterid, tweet, city, country, user, createdat, hashtagkeys, source, retweets, userstatuscount)" % (tweetTitle, emotion)
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    public_tweets_list = []
    for i in range(10):
        if emotion == "positive":
            for searchTerm in searchTerms[0]:
                public_tweets = api.search(q=searchTerm, lang="en", count="100")
                public_tweets_list.append(public_tweets)
        if emotion == "negative":
            for searchTerm in searchTerms[1]:
                public_tweets = api.search(q=searchTerm, lang="en", count="100")
                public_tweets_list.append(public_tweets)
    
    for public_tweets in public_tweets_list:
        for tweet in public_tweets:
            cleanTweet = cleanUpTweet(tweet.text)
            city, country = cleanUpPlace(tweet.user.location)
            username = cleanUpTweet(tweet.user.name)
            createdat = str(tweet.created_at.date())
            hashtags = tweet.entities['hashtags']
            hashtags = cleanHashtags(hashtags)
            source = tweet.source
            retweets = tweet.retweet_count
            userstatus = tweet.user.statuses_count
            
            session.execute(query, (tweet.id, cleanTweet.encode('utf-8'), city, country, username, createdat, hashtags, source, retweets, userstatus))
            
            

positive
negative


In [104]:
# Query 1
for emotion in positiveNegative:
    print("\n... Checking the conents of {} table".format(emotion))
    query = 'SELECT * FROM sentiment_%s_%s limit 2' % (tweetTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print(user_row)


... Checking the conents of positive table
Row(twitterid=1390766965292101633, tweet=u" dw_europe: EU leaders are meeting in Portugal to discuss the bloc's social affairs strategy against the backdrop of the #COVID19 pandem...", city=u'unknown', country=u'unknown', user=u'JM Hamilton', hashtagkeys=u'COVID19', source=u'Twitter Web App', retweets=9, userstatuscount=621164, createdat=u'2021-05-07')
Row(twitterid=1390759589310865415, tweet=u" DrAmbrishMithal: Don't chase #PCest after recoveryRepeat #PCR NOT necessary&gt;10 days after onset of symptoms&gt;3 days after absen...", city=u'Noida', country=u'India', user=u'Nikhil katiyar', hashtagkeys=u'RTPCRTest,RTPCR', source=u'Twitter for iPhone', retweets=127, userstatuscount=494, createdat=u'2021-05-07')

... Checking the conents of negative table
Row(twitterid=1390610581359562756, tweet=u' foogatwo: #NJ #COVID19 Case update for 5/5 Mass Deletions continuing Groundhog day Growing increasingly frustrated with inability of...', city=u'unknown

In [106]:
# Query 2: Find out the maximum number of times a particular hashtag has been used and the user 
# which has maximum retweets with that keyword in both the positive and negative table
for emotion in positiveNegative:
    print emotion
    query = "SELECT max(retweets), max(user) FROM sentiment_%s_%s where hashtagkeys='COVID19' ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print(user_row)

positive
Row(system_max_retweets=1132, system_max_user=u'sic')
negative
Row(system_max_retweets=1394, system_max_user=u'stay blazin')


In [107]:
# Query 3: Finding distinct values based on country
for emotion in positiveNegative:
    print '\n',emotion
    query = "SELECT DISTINCT twitterid, tweet, city, country, user, hashtagkeys, source, retweets, userstatuscount \
    FROM sentiment_%s_%s where country='India' LIMIT 2 ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print(user_row)


positive
Row(twitterid=1390759589310865415, tweet=u" DrAmbrishMithal: Don't chase #PCest after recoveryRepeat #PCR NOT necessary&gt;10 days after onset of symptoms&gt;3 days after absen...", city=u'Noida', country=u'India', user=u'Nikhil katiyar', hashtagkeys=u'RTPCRTest,RTPCR', source=u'Twitter for iPhone', retweets=127, userstatuscount=494)
Row(twitterid=1390777601547931648, tweet=u' sanjuydv: Shamefully Bihar stands at 8th rank in wastage of life saving Covid19 vaccines in IndiaMerely 766 lakh out of 13 Cr hv be...', city=u'Katihar', country=u'India', user=u'Suraj kumar', hashtagkeys=u'unknown', source=u'Twitter for Android', retweets=854, userstatuscount=533)

negative
Row(twitterid=1390688541802258436, tweet=u'shalmida drakchaurasia Sir He is in depressionFrustrated guyIgnore himMay God bless him with Covid19 and...', city=u'Ahmedabad', country=u'India', user=u'KUNDALINIYOGA #BabubhaiThakkar', hashtagkeys=u'unknown', source=u'Twitter for Android', retweets=0, userstatuscount=3110

In [221]:
# Query 4: Finding number of people tweeting positive and negative stuff from two different countries
for emotion in positiveNegative:
    print emotion
    query = "SELECT count(country) as Indians FROM sentiment_%s_%s where country='India' ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print(user_row)
        
    query = "SELECT count(country) as Canadians FROM sentiment_%s_%s where country='Canada' ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
    for user_row in rows:
        print(user_row)

positive
Row(indians=13)
Row(canadians=8)
negative
Row(indians=3)
Row(canadians=3)


In [222]:
# Query 5: Finding the number of tweets from a particular platform
for emotion in positiveNegative:
    print emotion
    query = "SELECT count(*) FROM sentiment_%s_%s where source='Twitter Web App' ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
#     print(rows)
    for user_row in rows:
        print(user_row)

positive
Row(count=65)
negative
Row(count=53)


In [223]:
# Query 6: Finding number of tweets with retweets > 25
for emotion in positiveNegative:
    print emotion
    query = "SELECT count(*) FROM sentiment_%s_%s where retweets>25 ALLOW FILTERING" % (tweetTitle, emotion)
    rows = session.execute(query)
#     print(rows)
    for user_row in rows:
        print(user_row)

positive
Row(count=80)
negative
Row(count=124)


In [224]:
# # Which countries are most people twitting from?
# for emotion in positiveNegative:
#     print emotion
#     query = "SELECT country, hashtagkeys FROM sentiment_%s_%s group by country ALLOW FILTERING" % (tweetTitle, emotion)
#     rows = session.execute(query)
# #     print(rows)
#     for user_row in rows:
#         print(user_row)

#### Creating a spark session which is connected to Cassandra and counting the number of rows in each.

In [225]:
countTokens = udf(lambda words: len(words), IntegerType())

# spark = SparkSession.builder.appName('demo').master("local").getOrCreate()
spark = SparkSession.builder.appName('demo').master("dse://dse:9042").getOrCreate()

tableNamePos = "sentiment_%s_positive" % (tweetTitle.lower())
tableNameNeg = "sentiment_%s_negative" % (tweetTitle.lower())

tablepos = spark.read.format("org.apache.spark.sql.cassandra").options(table=tableNamePos, keyspace="sentimentanalytics").load()
tableneg = spark.read.format("org.apache.spark.sql.cassandra").options(table=tableNameNeg, keyspace="sentimentanalytics").load()

print "Postive Table Count: "
print tablepos.count()
print "Negative Table Count: "
print tableneg.count()
print(tablepos)
display(tablepos)

Postive Table Count: 
220
Negative Table Count: 
209
DataFrame[twitterid: bigint, tweet: string, city: string, country: string, user: string, hashtagkeys: string, source: string, retweets: int, userstatuscount: int, createdat: string]


DataFrame[twitterid: bigint, tweet: string, city: string, country: string, user: string, hashtagkeys: string, source: string, retweets: int, userstatuscount: int, createdat: string]

In [226]:
# import pandas as pd
# df = tablepos.toPandas()
# df.to_csv('/project_folder/allTweetScores_s.csv', index=False)
# tablepos.write.format("csv").save('/project_folder/allTweetScores_s.csv')

#### Sentence to words tokenizer

In [227]:
tokenizerPos = Tokenizer(inputCol="tweet", outputCol="tweetwords")
tokenizedPos = tokenizerPos.transform(tablepos)
dfPos = tokenizedPos.select("tweetwords", "city", "country", "createdat", "hashtagkeys", "source", "retweets", "userstatuscount").withColumn("tokens", countTokens(col("tweetwords")))
showDF(dfPos)

tokenizerNeg = Tokenizer(inputCol="tweet", outputCol="tweetwords")
tokenizedNeg = tokenizerNeg.transform(tableneg)
dfNeg = tokenizedNeg.select("tweetwords", "city", "country", "createdat", "hashtagkeys","source", "retweets", "userstatuscount").withColumn("tokens", countTokens(col("tweetwords")))
showDF(dfNeg)

print(dfPos.count())
print(dfNeg.count())

,tweetwords,city,country,createdat,hashtagkeys,source,retweets,userstatuscount,tokens
0,"[, binancebcf:, #binance, charity's, year-long...",unknown,unknown,2021-05-07,Binance,Twitter for iPhone,4,617,18
1,"[, cdcgov:, at, the, start, of, 2021, people, ...",unknown,unknown,2021-05-07,COVID19,Twitter Web App,37,186560,24
2,"[, wef:, two, key, conditions, for, a, post-pa...",unknown,unknown,2021-05-07,"covid19,economics",Twitter for iPad,12,15526,13
3,"[#corona, info, for, #usa:new, cases:, 27163to...",Karlsruhe,Germany,2021-05-07,"Corona,USA",corona-tracker-app,0,13927,14
4,"[, ethicalsid:, one, can, feel, in, saurabh_ml...",Dubai,unknown,2021-05-07,COVID19,Twitter for iPhone,151,2770,23


,tweetwords,city,country,createdat,hashtagkeys,source,retweets,userstatuscount,tokens
0,"[05/07/2021, update:people, tested:, 905825, (...",unknown,unknown,2021-05-07,unknown,ri_covid_19,0,315,11
1,"[, jkwan_md:, may, 7:, #covid19, in, #ontario3...",unknown,Canada,2021-05-07,"COVID19,Ontario",Twitter Web App,102,348030,15
2,"[robking65, natwittee, ctvnews, none, it's, a,...",Toronto,Canada,2021-05-07,unknown,Twitter for Android,0,1540,17
3,"[#alberta, restaurant, owners, frustrated, as,...",unknown,unknown,2021-05-06,"Alberta,COVID19",Twitter for iPhone,0,43243,14
4,"[look, what, u, voted, 4, a, man, without, an,...",unknown,unknown,2021-05-07,unknown,Twitter for Android,2,232906,21


220
209


#### Removing stop words

In [228]:
removerPos = StopWordsRemover(inputCol="tweetwords", outputCol="tweetnostopwords")
removedPos = removerPos.transform(dfPos)
dfPosStop = removedPos.select("tweetwords", "tweetnostopwords", "city", "country", "createdat", "hashtagkeys","source", "retweets", "userstatuscount").withColumn("tokens", countTokens(col("tweetwords"))).withColumn("notokens", countTokens(col("tweetnostopwords")))
showDF(dfPosStop)

removerNeg = StopWordsRemover(inputCol="tweetwords", outputCol="tweetnostopwords")
removedNeg = removerNeg.transform(dfNeg)
dfNegStop = removedNeg.select("tweetwords", "tweetnostopwords", "city", "country", "createdat", "hashtagkeys","source", "retweets", "userstatuscount").withColumn("tokens", countTokens(col("tweetwords"))).withColumn("notokens", countTokens(col("tweetnostopwords")))
showDF(dfNegStop)

print(dfPosStop.count())
print(dfNegStop.count())

,tweetwords,tweetnostopwords,city,country,createdat,hashtagkeys,source,retweets,userstatuscount,tokens,notokens
0,"[, binancebcf:, #binance, charity's, year-long...","[, binancebcf:, #binance, charity's, year-long...",unknown,unknown,2021-05-07,Binance,Twitter for iPhone,4,617,18,15
1,"[, cdcgov:, at, the, start, of, 2021, people, ...","[, cdcgov:, start, 2021, people, 65+, half, #c...",unknown,unknown,2021-05-07,COVID19,Twitter Web App,37,186560,24,12
2,"[, wef:, two, key, conditions, for, a, post-pa...","[, wef:, two, key, conditions, post-pandemic, ...",unknown,unknown,2021-05-07,"covid19,economics",Twitter for iPad,12,15526,13,11
3,"[#corona, info, for, #usa:new, cases:, 27163to...","[#corona, info, #usa:new, cases:, 27163today, ...",Karlsruhe,Germany,2021-05-07,"Corona,USA",corona-tracker-app,0,13927,14,13
4,"[, ethicalsid:, one, can, feel, in, saurabh_ml...","[, ethicalsid:, one, feel, saurabh_mlagk's, vo...",Dubai,unknown,2021-05-07,COVID19,Twitter for iPhone,151,2770,23,14


,tweetwords,tweetnostopwords,city,country,createdat,hashtagkeys,source,retweets,userstatuscount,tokens,notokens
0,"[05/07/2021, update:people, tested:, 905825, (...","[05/07/2021, update:people, tested:, 905825, (...",unknown,unknown,2021-05-07,unknown,ri_covid_19,0,315,11,11
1,"[, jkwan_md:, may, 7:, #covid19, in, #ontario3...","[, jkwan_md:, may, 7:, #covid19, #ontario3166,...",unknown,Canada,2021-05-07,"COVID19,Ontario",Twitter Web App,102,348030,15,14
2,"[robking65, natwittee, ctvnews, none, it's, a,...","[robking65, natwittee, ctvnews, none, fair, qu...",Toronto,Canada,2021-05-07,unknown,Twitter for Android,0,1540,17,14
3,"[#alberta, restaurant, owners, frustrated, as,...","[#alberta, restaurant, owners, frustrated, #co...",unknown,unknown,2021-05-06,"Alberta,COVID19",Twitter for iPhone,0,43243,14,11
4,"[look, what, u, voted, 4, a, man, without, an,...","[look, u, voted, 4, man, without, ounce, consc...",unknown,unknown,2021-05-07,unknown,Twitter for Android,2,232906,21,14


220
209


### Finding the positive, negative and neutral tweets

#### Negative Tweets

In [229]:
pandaNeg = dfNegStop.toPandas()
sentimentScoreNeg = 0
countNeg = 0
numTweets = 0
negList = list()
netralList = list()

for index, row in pandaNeg.iterrows():
    if positive(row["tweetnostopwords"], .1):
        countNeg = countNeg + 1
    scoreNeg = sentiment(row['tweetnostopwords'])[0]
    if scoreNeg <= 0:
        negList.append((row['country'], row['city'], row['createdat'], row['hashtagkeys'], sentiment(row["tweetnostopwords"]), positive(row["tweetnostopwords"]), \
                        sentiment(row['tweetnostopwords']).assessments, row['source'], row['retweets'], row['userstatuscount'], scoreNeg))
        sentimentScoreNeg = scoreNeg + sentimentScoreNeg
    else:
        netralList.append((row['country'], row['city'], row['createdat'], row['hashtagkeys'], sentiment(row["tweetnostopwords"]), positive(row["tweetnostopwords"]), \
                           sentiment(row['tweetnostopwords']).assessments, row['source'], row['retweets'], row['userstatuscount'], scoreNeg))

labels = ['Country', 'City', 'Create_At', 'Hashtags Keyword', 'Sentiment Score', 'Postive', 'Assessments', 'Source', 'Retweets', 'User Status', 'sentiment score']
negativeTweetScores = pandas.DataFrame.from_records(negList, columns=labels)
negativeTweetScores

,Country,City,Create_At,Hashtags Keyword,Sentiment Score,Postive,Assessments,Source,Retweets,User Status,sentiment score
0,unknown,unknown,2021-05-07,unknown,"(0.0, 0.0)",False,[],ri_covid_19,0,315,0.000000
1,Canada,unknown,2021-05-07,"COVID19,Ontario","(0.0, 0.0)",False,[],Twitter Web App,102,348030,0.000000
2,unknown,unknown,2021-05-06,"Alberta,COVID19","(-0.7, 0.2)",False,"[([frustrated], -0.7, 0.2, None)]",Twitter for iPhone,0,43243,-0.700000
3,unknown,unknown,2021-05-07,unknown,"(0.0, 0.0)",False,[],Twitter for Android,2,232906,0.000000
4,India,Chennai,2021-05-07,unknown,"(0.0, 0.0)",False,[],Twitter for Android,89,4344,0.000000
5,unknown,unknown,2021-05-07,COVID19,"(0.0, 0.0)",False,"[([fourth], 0.0, 0.0, None)]",Twitter Web App,13,9743,0.000000
6,unknown,unknown,2021-05-06,"NJ,COVID19","(-0.7, 0.2)",False,"[([frustrated], -0.7, 0.2, None)]",Twitter for Android,76,15515,-0.700000
7,unknown,unknown,2021-05-07,unknown,"(0.0, 0.0)",False,[],Twitter for Android,98,3320,0.000000
8,unknown,unknown,2021-05-06,unknown,"(0.0, 0.0)",False,[],Twitter for Android,0,8,0.000000
9,United Kingdom,unknown,2021-05-07,unknown,"(0.0, 0.0666666666667)",False,"[([last], 0.0, 0.0666666666667, None)]",Twitter for iPad,0,30774,0.000000


#### Positive Tweet

In [230]:
pandaPos = dfPosStop.toPandas()
sentimentScore = 0
countPos = 0
poslist = list()

for index, row in pandaPos.iterrows():
    if not positive(row["tweetnostopwords"]) and sentiment(row["tweetnostopwords"])[0] != 0.0:
        countPos = countPos + 1
    score = sentiment(row['tweetnostopwords'])[0]
    if score > 0:
        poslist.append((row['country'],row['city'], row['createdat'], row['hashtagkeys'], sentiment(row["tweetnostopwords"]), positive(row["tweetnostopwords"]), \
                         sentiment(row['tweetnostopwords']).assessments, row['source'], row['retweets'], row['userstatuscount'], score))
        sentimentScore = score + sentimentScore
    else:
        netralList.append((row['country'], row['city'], row['createdat'], row['hashtagkeys'], sentiment(row["tweetnostopwords"]), positive(row["tweetnostopwords"]), \
                           sentiment(row['tweetnostopwords']).assessments, row['source'], row['retweets'], row['userstatuscount'], score))
        
postiveTweetScores = pandas.DataFrame.from_records(poslist, columns=labels)
postiveTweetScores

,Country,City,Create_At,Hashtags Keyword,Sentiment Score,Postive,Assessments,Source,Retweets,User Status,sentiment score
0,unknown,unknown,2021-05-07,Binance,"(0.116666666667, 0.133333333333)",True,"[([social], 0.0333333333333, 0.0666666666667, ...",Twitter for iPhone,4,617,0.116667
1,unknown,unknown,2021-05-07,"covid19,economics","(0.1, 0.6)",True,"[([key], 0.0, 1.0, None), ([economic], 0.2, 0....",Twitter for iPad,12,15526,0.100000
2,unknown,Dubai,2021-05-07,COVID19,"(0.103703703704, 0.477777777778)",True,"[([tough], -0.388888888889, 0.833333333333, No...",Twitter for iPhone,151,2770,0.103704
3,Canada,Toronto,2021-05-07,"Toronto,COVID19","(0.136363636364, 0.5)",True,"[([live], 0.136363636364, 0.5, None)]",Twitter for Android,7,2182,0.136364
4,unknown,unknown,2021-05-07,COVID19,"(0.25, 0.333333333333)",True,"[([first], 0.25, 0.333333333333, None)]",Twitter Web App,1,1857,0.250000
5,unknown,Seattle,2021-05-07,HugAHesitant,"(0.6, 0.9)",True,"[([amazing], 0.6, 0.9, None)]",Twitter Web App,0,4085,0.600000
6,unknown,unknown,2021-05-07,unknown,"(0.166666666667, 0.333333333333)",True,"[([older], 0.166666666667, 0.333333333333, None)]",Twitter for Android,1,13377,0.166667
7,unknown,unknown,2021-05-07,COVID19,"(0.518181818182, 0.677272727273)",True,"[([incredibly], 0.9, 0.9, None), ([new], 0.136...",Twitter for iPhone,35,128843,0.518182
8,India,Hyderabad,2021-05-07,COVID19,"(0.35, 0.65)",True,"[([beloved], 0.7, 1.0, None), ([personally], 0...",Twitter for Android,79,91,0.350000
9,unknown,unknown,2021-05-07,unknown,"(0.318181818182, 0.527272727273)",True,"[([love], 0.5, 0.6, None), ([new], 0.136363636...",Twitter for Android,14,576,0.318182


#### Neutral Tweets and concatenating and saving all 3 dataframes into one csv

In [231]:
neutralTweetScores = pandas.DataFrame.from_records(nutralList, columns=labels)
neutralTweetScores

,Country,City,Create_At,Hashtags Keyword,Sentiment Score,Postive,Assessments,Source,Retweets,User Status,sentiment score
0,Canada,Toronto,2021-05-07,unknown,"(0.166666666667, 0.533333333333)",True,"[([fair], 0.7, 0.9, None), ([anger], -0.7, 0.2...",Twitter for Android,0,1540,0.166667
1,unknown,unknown,2021-05-07,COVID19,"(0.466666666667, 0.716666666667)",True,"[([easily], 0.433333333333, 0.833333333333, No...",Twitter for iPhone,398,64373,0.466667
2,unknown,unknown,2021-05-07,unknown,"(0.3, 0.2)",True,"[([fun], 0.3, 0.2, None)]",Twitter for Android,2,74,0.300000
3,unknown,unknown,2021-05-07,unknown,"(0.0833333333333, 0.55)",False,"[([fresh], 0.3, 0.5, None), ([active], -0.1333...",Twitter for Android,18,242872,0.083333
4,unknown,Leeds,2021-05-07,COVID19,"(0.333333333333, 0.5)",True,"[([pregnant], 0.333333333333, 0.5, None)]",Twitter for Android,291,215,0.333333
5,unknown,unknown,2021-05-07,COVID19,"(0.325, 0.7)",True,"[([huge], 0.4, 0.9, None), ([higher], 0.25, 0....",Twitter for Android,1394,56104,0.325000
6,unknown,unknown,2021-05-07,COVID19,"(0.325, 0.7)",True,"[([huge], 0.4, 0.9, None), ([higher], 0.25, 0....",Twitter Web App,1394,23493,0.325000
7,unknown,Garfield Heights,2021-05-07,COVID19,"(0.466666666667, 0.716666666667)",True,"[([easily], 0.433333333333, 0.833333333333, No...",Twitter Web App,398,81643,0.466667
8,France,Paris,2021-05-07,COVID19,"(0.325, 0.7)",True,"[([huge], 0.4, 0.9, None), ([higher], 0.25, 0....",Twitter for Android,1394,16584,0.325000
9,unknown,Texas,2021-05-06,COVID19,"(0.466666666667, 0.716666666667)",True,"[([easily], 0.433333333333, 0.833333333333, No...",Twitter for Android,398,138651,0.466667


In [254]:
df = pandas.concat([negativeTweetScores, postiveTweetScores, neutralTweetScores])
csv_name = '/project_folder/allTweetSentimentsfinal.csv'
df.to_csv(csv_name, index = False, header=True, encoding='utf-8-sig')

In [258]:
df = pandas.read_csv(csv_name)
final_dict = {}
for i, group in df.groupby(['Country']):
    # print(group)
    # break
    p_grp = group[group['Postive'].apply(lambda x: x == True)]
    n_grp = group[group['Postive'].apply(lambda x: x == False)]

    if i not in final_dict:
        final_dict[i] = {}
    pos = len(p_grp)
    neg = len(n_grp)
    pos_key = 'positive_count'
    if pos_key not in final_dict[i]:
        final_dict[i][pos_key] = pos
    neg_key = 'negative_count'
    if neg_key not in final_dict[i]:
        final_dict[i][neg_key] = neg

    sources = list(set(group.Source))
    num_sources = len(sources)
    s_key = 'sources'
    n_s_key = 'num_sources'
    if s_key not in final_dict[i]:
        final_dict[i][s_key] = sources
    if n_s_key not in final_dict[i]:
        final_dict[i][n_s_key] = num_sources

    p_hashtags = list(set(group['Hashtags Keyword']))
    p_all_hashtags = []
    for hashtag in p_hashtags:
        for x in hashtag.split(','):
            if x.strip() != 'unknown':
                p_all_hashtags.append(x.strip())
    p_hashtag_key = 'hashtags'
    if p_hashtag_key not in final_dict[i]:
        final_dict[i][p_hashtag_key] = list(set(p_all_hashtags))
    n_p_hashtag_key = 'num_hashtags'
    if n_p_hashtag_key not in final_dict[i]:
        final_dict[i][n_p_hashtag_key] = len(set(p_all_hashtags))

    clean_words = group.Assessments.apply(
        lambda x: re.findall(r"\[([A-Za-z0-9_]+)\]", x))
    all_clean_words = [y for x in clean_words for y in x]
    # all_clean_words = []
    # for x in clean_words:
    #     for y in x:
    #         all_clean_words.append(y)
    # print(all_clean_words)

    s_a_key = 'sentiment_assessment_words'
    if s_a_key not in final_dict[i]:
        final_dict[i][s_a_key] = all_clean_words

In [260]:
pandas.DataFrame.from_dict(final_dict).T

,hashtags,negative_count,num_hashtags,num_sources,positive_count,sentiment_assessment_words,sources
Argentina,"[economics, covid19]",0,2,1,1,"[key, economic]",[Twitter Web App]
Australia,[COVID19],0,1,2,2,"[older, easily, love]","[Twitter for iPhone, Twitter for Android]"
Canada,"[Toronto, Ontario, COVID19]",2,3,3,3,"[live, live, fair, anger, many]","[Twitter for iPhone, Twitter Web App, Twitter ..."
France,"[COVID19, Massachusetts, Covid19]",1,3,2,1,"[last, huge, higher]","[#COVID19, Twitter for Android]"
Georgia,[COVID19],0,1,1,2,"[easily, love, easily, love]",[Twitter for iPad]
Germany,"[USA, Corona]",1,2,1,0,[active],[corona-tracker-app]
Greece,"[coronavirus, Greece]",0,2,1,1,"[new, confirmed]",[Twitter for Android]
India,"[COVID19, Covid19]",3,2,2,4,"[beloved, personally, positive, positive, posi...","[SocialPilot.co, Twitter for Android]"
Ireland,[COVID19],0,1,3,3,"[incredibly, new, spanish, economic, new, old]","[Twitter for iPad, Twitter Web App, Twitter fo..."
Japan,"[Binance, COVID19, JAPAN]",0,3,2,2,"[social, economic, many]","[Twitter for iPhone, TweetDeck]"
